In [23]:
import openai
import os

In [30]:
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')
#MODEL="gpt-3.5-turbo"
MODEL="gpt-4"

In [31]:
def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

[](../images/03-Principle1.png)

![Principle 1](../images/03-Principle1.png)

In [32]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Provide clear and specific instructions to guide a model towards the desired output, as longer prompts often offer more clarity and context, leading to more detailed and relevant results.
